In [1]:
import polars as pl
from pprint import pprint
import pandas as pd

In [53]:
df_pd=pd.read_json('ShareGPT_V3_unfiltered_cleaned_split_no_imsorry.json')

In [59]:
df=pl.read_json('ShareGPT_V3_unfiltered_cleaned_split_no_imsorry.json')

In [50]:
df.estimated_size("mb")

522.9341344833374

In [25]:
df.shape

(94145, 5)

In [54]:
# Take the first part of the struct as the questionner
df=df.with_columns(pl.col('conversations').list.first().alias('human')
                   # Unnest the struct and rename it
                ).unnest('human').rename({'from':'questionner', 'value':'question'}
                                         # Take the last part as the answerer
                                         ).with_columns(pl.col('conversations').list.last().alias('gpt')
                                                        # unnest and rename it
                                                        ).unnest('gpt').drop('conversations').rename({'from':'answerer', 'value': 'answer'})
df.head()

id,questionner,question,answerer,answer
str,str,str,str,str
"""QWJhYvA_0""","""human""","""Summarize the …","""gpt""","""Tony Robbins d…"
"""i6IyJda_0""","""human""","""How to tell if…","""gpt""","""1. Homogeneity…"
"""A5AbcES_0""","""human""","""In Java, I wan…","""gpt""","""You can use th…"
"""hRPPgZT_0""","""human""","""Metaphorical l…","""gpt""","""Lo and behold!…"
"""hRPPgZT_11""","""gpt""","""Lo and behold!…","""gpt""","""Behold, fellow…"


In [55]:
df_pd_sec=df_pd.explode('conversations')
df_conversations = pd.json_normalize(df_pd_sec['conversations'])

# separate the questions and answers
df_questions = df_conversations[df_conversations['from'] == 'human'].rename(columns={'from': 'questionner', 'value': 'question'})
df_answers = df_conversations[df_conversations['from'] == 'gpt'].rename(columns={'from': 'answerer', 'value': 'answer'})

# after, reset indices to join them properly
df_questions = df_questions.reset_index(drop=True)
df_answers = df_answers.reset_index(drop=True)

# and then, join questions and answers based on the indices
df_bis = pd.concat([df_questions[['questionner', 'question']], df_answers[['answerer', 'answer']]], axis=1)
df_bis=pd.concat([df_pd['id'], df_bis], axis=1)
df_bis=df_bis.reset_index(drop=True)
df_bis.head()

,id,questionner,question,answerer,answer
0,QWJhYvA_0,human,Summarize the main ideas of Jeff Walker's Prod...,gpt,Here are the main ideas of Jeff Walker's Produ...
1,i6IyJda_0,human,Summarize the main ideas of Brendon Burchard's...,gpt,Here are the main ideas of Brendon Burchard's ...
2,A5AbcES_0,human,What are the mental triggers in Jeff Walker's ...,gpt,Here are the mental triggers described in Jeff...
3,hRPPgZT_0,human,Write a summary of why scarcity and urgency ar...,gpt,Scarcity and urgency are two of the strongest ...
4,hRPPgZT_11,human,Summarize Russell Brunson's Perfect Webinar Sc...,gpt,Russell Brunson's Perfect Webinar Script is a ...


In [58]:
# Just take questions under 5 words
df=df.with_columns(pl.col('question').str.split(' ').alias('number')
                ).filter(pl.col('number').list.len()<5
                         #Replace everything that is not alpha-numeric by spaces and drop nulls
                         ).drop('number'
                                ).with_columns(pl.col('question', 'answer').str.replace_all(r'[^a-zA-Z]', ' ')).drop_nulls()
df.head()

id,questionner,question,answerer,answer
str,str,str,str,str
"""VY7cMKG_0""","""human""","""Poly Ether Eth…","""gpt""","""Here are the t…"
"""ng7rjf6_0""","""human""","""explane more""","""gpt""","""Sure Here s a…"
"""W4wL13P_0""","""human""","""explain the me…","""gpt""","""The metaverse …"
"""v4PzAY8_0""","""human""","""Hello""","""gpt""","""Hello How can…"
"""mxEe31E_0""","""human""","""Explain deposi…","""gpt""","""Deposition and…"


In [57]:
df_bis['number'] = df_bis['question'].str.split(' ')
df_bis = df_bis[df_bis['number'].apply(lambda x: len(x) < 5 if isinstance(x, list) else False)]
df_bis = df_bis.drop(columns=['number'])

# Replace non-alphabetic characters in 'question' and 'answer' columns and drop nulls
df_bis['question'] = df_bis['question'].str.replace(r'[^a-zA-Z]', ' ', regex=True)
df_bis['answer'] = df_bis['answer'].str.replace(r'[^a-zA-Z]', ' ', regex=True)

df_bis = df_bis.dropna()
df_bis.head()


,id,questionner,question,answerer,answer
9,VY7cMKG_0,human,continue explain it,gpt,The language used to describe the addressing m...
10,H1aZlSE_0,human,more perplexity,gpt,Behold the arcane art of metaphorical languag...
11,wNBG8Gp_0,human,more perplexity,gpt,Lo and behold The arcane and elusive art of m...
14,wNBG8Gp_33,human,More verbose and details,gpt,Lo and behold By the grace of divine interven...
16,wNBG8Gp_65,human,double the length,gpt,Prepare to be dazzled by the sheer magnificenc...
